In [5]:
import numpy as np

# Constants
k_e = 8.9875517873681764e9  # Coulomb's constant in N m²/C²
fusion_energy_threshold = 1e12  # Example threshold for fusion in joules

class Collider:
    def __init__(self, particles):
        self.particles = particles

    def detect_collisions(self):
        """Detect collisions based on proximity between particles."""
        collision_threshold = 1e-10  # Distance threshold for collision detection
        for i, p1 in enumerate(self.particles):
            for j, p2 in enumerate(self.particles[i+1:], start=i+1):
                distance = np.linalg.norm(p1.position - p2.position)
                if distance < collision_threshold:
                    self.handle_collision(p1, p2)

    def handle_collision(self, p1, p2):
        """Handle a collision between two particles, with conservation laws and potential fusion or transformation."""
        # Initial energy and momentum for conservation checks
        total_initial_momentum = p1.mass * p1.velocity + p2.mass * p2.velocity
        total_initial_energy = p1.total_energy() + p2.total_energy()
        total_charge = p1.charge + p2.charge  # Ensure charge conservation

        # Check if fusion occurs based on total energy
        if total_initial_energy > fusion_energy_threshold:
            self.perform_fusion(p1, p2, total_initial_momentum, total_charge)
        else:
            # If no fusion, perform a standard collision with momentum and energy conservation
            self.apply_momentum_conservation(p1, p2)

            # Implement stochastic transformation (Monte Carlo)
            self.stochastic_transformation(p1, p2)

    def apply_momentum_conservation(self, p1, p2):
        """Apply conservation of momentum and energy during elastic collision."""
        # Calculate final velocities using conservation of momentum and energy
        if p1.mass != p2.mass:
            v1_final = ((p1.mass - p2.mass) / (p1.mass + p2.mass)) * p1.velocity + (2 * p2.mass / (p1.mass + p2.mass)) * p2.velocity
            v2_final = ((p2.mass - p1.mass) / (p1.mass + p2.mass)) * p2.velocity + (2 * p1.mass / (p1.mass + p2.mass)) * p1.velocity
            p1.velocity = v1_final
            p2.velocity = v2_final
        else:
            # If masses are equal, swap velocities
            p1.velocity, p2.velocity = p2.velocity, p1.velocity

    def stochastic_transformation(self, p1, p2):
        """Randomly transform particles post-collision based on a probability."""
        transformation_probability = 0.3  # Example probability for transformation

        if np.random.rand() < transformation_probability:
            # Example of splitting a particle into two new particles
            new_mass = p1.mass * 0.5
            p1.mass *= 0.5
            p2.mass *= 0.5
            p1.type = "DecayProduct1"
            p2.type = "DecayProduct2"
            print(f"Particles transformed into new types {p1.type} and {p2.type} with stochastic transformation.")

    def perform_fusion(self, p1, p2, total_initial_momentum, total_charge):
        """Fuse two particles into a single new particle with combined mass and charge."""
        combined_mass = p1.mass + p2.mass
        additional_mass = ((p1.total_energy() + p2.total_energy()) - (combined_mass * c**2)) / c**2
        new_mass = combined_mass + additional_mass
        new_velocity = total_initial_momentum / new_mass
        fusion_particle = Particle(position=p1.position, velocity=new_velocity, mass=new_mass, charge=total_charge, p_type="FusionParticle")
        self.particles.append(fusion_particle)
        print(f"Fusion created a new particle with mass {new_mass} and charge {total_charge}.")

    def calculate_electric_force(self, p1, p2):
        """Calculate the electric force between two charged particles using Coulomb's law."""
        distance_vector = p2.position - p1.position
        distance = np.linalg.norm(distance_vector)
        if distance == 0:
            return np.zeros(3)  # Avoid division by zero if particles overlap
        force_magnitude = k_e * (p1.charge * p2.charge) / distance**2
        force_direction = distance_vector / distance
        return force_magnitude * force_direction

    def apply_forces(self):
        """Apply electric forces between all particle pairs and update velocities accordingly."""
        time_step = 0.01
        for i, p1 in enumerate(self.particles):
            for p2 in self.particles[i+1:]:
                electric_force = self.calculate_electric_force(p1, p2)
                p1_acceleration = electric_force / p1.mass
                p2_acceleration = -electric_force / p2.mass
                p1.velocity += p1_acceleration * time_step
                p2.velocity += p2_acceleration * time_step